## Running LOCAL spark to read and write to HDFS

Local spark runs the drivers and executors in the same JVM. As such it has access to the Kerberos ticket in the cache. 

Hence we can skip the task of producing a HDFS Delegation token which is produced internally. Local Spark can read write to HDFS without any extra development similar to how we can do the same with EMR Spark (with driver running in client mode) as long as the driver runs inside a workspace with a valid Kerberos ticket (you can examine it by running `kilist` in the terminal)

In [ ]:
!klist

In [ ]:
import os

%env HDFS_PATH /user/dominospark/small_data
%env HADOOP_HOME /usr/lib/hadoop
%env HADOOP_CONF_DIR /etc/hadoop/conf
%env HADOOP_YARN_HOME /usr/lib/hadoop
%env HADOOP_MAPRED_HOME /usr/lib/hadoop
%env HADOOP_HDFS_HOME /usr/lib/hadoop-hdfs

%env SPARK_HOME /usr/lib/spark
%env SPARK_CONF_DIR /etc/spark/conf
#%env PYTHONPATH /opt/spark/python/lib/py4j-0.10.7-src.zip
%env PYSPARK_PYTHON /opt/conda/bin/python
%env PYSPARK_DRIVER_PYTHON /opt/conda/bin/python
hdfs_endpoint=os.environ['HDFS_ENDPOINT']

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, IntegerType
import random

In [ ]:
sparkSession = SparkSession.builder.appName("example-pyspark-read-and-write") \
.master('local[*]') \
.config("fs.default.name", hdfs_endpoint) \
.getOrCreate()

In [ ]:
sc=sparkSession.sparkContext

In [ ]:
def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1


In [ ]:


columns = StructType([ StructField("name", StringType(), True),
                      StructField("value", DoubleType(), True)
                    ])

count = sc.parallelize(range(0, 1000),1) \
             .filter(inside).count()
data = [("Pi",4.0 * count/1000)]

df = sparkSession.createDataFrame(data=data, schema=columns)

df.show()

In [ ]:
#Let us write to a dataset
ds_path = '/user/dominospark/my-pi'
!/mnt/code/scripts/my_hdfs.sh dfs -rmr '/user/dominospark/my-pi*'
df.write.csv(ds_path)
#Read it back
sparkSession.read.csv(ds_path).show()


In [ ]:
!/mnt/code/scripts/my_hdfs.sh dfs -rmr '/user/dominospark/small-data-100/'
hdfs_src_path = '/user/dominospark/largedata/'
hdfs_dest_path =  '/user/dominospark/small-data-100/'
local_dest_path = 'file:///mnt/data/ON-DEMAND-SPARK/small-data-100'
!rm -rf /mnt/data/ON-DEMAND-SPARK/small-data-100
filter_criteria = 100
sparkSession = SparkSession.builder.appName("Generate Data") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("fs.default.name", hdfs_endpoint) \
    .getOrCreate()

    
columns = StructType([ StructField("id", IntegerType(), True), \
                       StructField("v1", IntegerType(), True),\
                       StructField("v2", IntegerType(), True),\
                       StructField("v3", IntegerType(), True) ])

df_load = sparkSession.read.csv(hdfs_src_path,columns)
df_load_filtered = df_load.where(df_load.id < filter_criteria)
df_load_filtered.write.csv(hdfs_dest_path)
df_load_filtered.write.csv(local_dest_path)


In [ ]:
df_load_read = sparkSession.read.csv(hdfs_dest_path,columns)
df_load_read.show()

In [ ]:
df_load_read = sparkSession.read.csv(local_dest_path,columns)
df_load_read.show()

In [ ]:
sparkSession.stop()

In [ ]:
!/mnt/code/scripts/my_hdfs.sh dfs -ls